In [10]:
from flask import Flask, request, jsonify

from flask_restplus import Resource, Api
from flask_sqlalchemy import SQLAlchemy

import sqlite3

import requests
import json
import io
import datetime
import uuid

In [11]:
def create_db(db_file):
    '''
    use this function to create a db, don't change the name of this function.
    db_file: Your database's name.
    '''
    pass 

In [12]:
def getJsonContent(indicator_id):
    url = 'http://api.worldbank.org/v2/countries/all/indicators/' + indicator_id + '?per_page=1000&date=2013:2018&format=json'

    response = requests.get(url)
    jsonContent = json.load(io.BytesIO(response.content))
    if indicator_id != 'favicon.ico' and jsonContent[0]['pages'] > 1:
        pages = jsonContent[0]['pages']

        for page in range(2,pages+1):
            url = 'http://api.worldbank.org/v2/countries/all/indicators/' + indicator_id + '?per_page=1000&date=2013:2018&format=json&page=' + str(page) 
            response = requests.get(url)
            jsonContent = json.load(io.BytesIO(response.content))
    return jsonContent

def cleanJSON(jsonContent):
    ret = dict()
    ret['indicator'] = jsonContent[1][0]['indicator']['id']
    ret['indicator_value'] = jsonContent[1][0]['indicator']['value']

    ret['entries'] = list()
    for item in jsonContent[1]:
        ret['entries'].append({'country':item['country']['value'], 'date':item['date'], 'value':item['value']})

    return ret

def _loadJSON(jsonContent, creation_time, collection_id):
    indicator = jsonContent['indicator']
    indicator_value = jsonContent['indicator_value']

    for item in jsonContent['entries']:
        nextItem = Indicator(creation_time = creation_time, 
                             collection_id = collection_id, 
                             indicator = indicator,
                             indicator_value = indicator_value,
                             country = item['country'],
                             date = item['date'],
                             value = item['value'])
        db.session.add(nextItem)
        
    db.session.commit()
    
def loadJSON(jsonContent, creation_time, collection_id):
    indicator = jsonContent['indicator']
    indicator_value = jsonContent['indicator_value']
    
    nextItem = Collection(creation_time = creation_time, 
                             collection_id = collection_id, 
                             indicator = indicator,
                             indicator_value = indicator_value)
    db.session.add(nextItem)

    for item in jsonContent['entries']:
        nextItem = Entry(    collection_id = collection_id, 
                             country = item['country'],
                             date = item['date'],
                             value = item['value'])
        db.session.add(nextItem)
        
    db.session.commit()

In [13]:
app = Flask(__name__)
# app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///memory:'
db = SQLAlchemy(app)

/opt/anaconda/lib/python3.6/site-packages/flask_sqlalchemy/__init__.py:774: UserWarning: Neither SQLALCHEMY_DATABASE_URI nor SQLALCHEMY_BINDS is set. Defaulting SQLALCHEMY_DATABASE_URI to "sqlite:///:memory:".
  'Neither SQLALCHEMY_DATABASE_URI nor SQLALCHEMY_BINDS is set. '
/opt/anaconda/lib/python3.6/site-packages/flask_sqlalchemy/__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [14]:
class Entry(db.Model):
    collection_id = db.Column(db.String(36), db.ForeignKey('collection.collection_id'), primary_key=True)
    country = db.Column(db.String(20), primary_key=True)
    date = db.Column(db.String(4), primary_key=True)
    value = db.Column(db.Float)
    
    @property
    def serialize(self):
        return {'country': self.country,
                'date': self.date,
                'value': self.value}

    def __repr__(self):
        return '<Country: {}, Date: {}, Value: {}>'.format(self.country, self.date, self.value)
    
class Collection(db.Model):
    collection_id = db.Column(db.String(36), primary_key=True)
    creation_time = db.Column(db.DateTime, nullable=False)
    indicator = db.Column(db.String(20), nullable=False)
    indicator_value = db.Column(db.String(20), nullable=False)
        
    @property
    def serialize(self):
        return {'collection_id': self.collection_id,
                'indicator': self.indicator,
                'indicator_value': self.indicator_value,
                'creation_time': self.creation_time}
    
    def get_repr(self, collection):
        return {"location" : "/" + collection + "/" + self.collection_id, 
                "collection_id" : self.collection_id,  
                "creation_time": self.creation_time.strftime('%B %d %Y - %H:%M:%S'),
                "indicator" : self.indicator}

    def __repr__(self):
        return str(self.get_repr)

In [15]:
db.create_all()
api = Api(app)

In [16]:
@api.route('/<collection>')
class Collection_Endpoint(Resource):
    def post(self, collection):
        indicator_id = api.payload['indicator_id']
        jsonContent = getJsonContent(indicator_id)
        cleanedContent = cleanJSON(jsonContent)
        
        creation_time = datetime.datetime.utcnow()
        collection_id = str(uuid.uuid4())

        loadJSON(cleanedContent, creation_time, collection_id)

        return Response(json.dumps({'location' : '/' + collection + '/' + collection_id, 
                'collection_id' : collection_id,  
                'creation_time': creation_time.strftime('%B %d %Y - %H:%M:%S'),
                'indicator' : indicator_id}), status=201, mimetype='application/json')
    
    def get(self, collection):
        return jsonify(list(x.get_repr(collection) for x in Collection.query.all()))


@api.route('/<collection>/<collection_id>')
class Collection_ID_Endpoint(Resource):
    def delete(self, collection, collection_id):
        db.session.query(Entry).filter(Entry.collection_id == collection_id).delete()
        db.session.query(Collection).filter(Collection.collection_id == collection_id).delete()
        db.session.commit()
        
        return jsonify({'message' :'Collection = ' + collection_id + 'is removed from the database!'})
    
    def get(self, collection, collection_id):
        entries = list(x.serialize for x in Entry.query.filter(Entry.collection_id == collection_id).all())
        collections = Collection.query.filter(Collection.collection_id == collection_id).all()[0].serialize
        
        collections['entries'] = entries
        
        return jsonify(collections)
    
# api.add_resource(Interface, '/<collection>')

In [17]:
@api.route('/<collection>/<collection_id>/<year>/<country>')
class Country_Year_Endpoint(Resource):
    def get(self, collection, collection_id, year, country):
        entries = list(x.serialize for x in Entry.query.filter(Entry.collection_id == collection_id,
                                                               Entry.country == country.title(),
                                                               Entry.date == year
                                                              ).all())
        collections = Collection.query.filter(Collection.collection_id == collection_id).all()[0].serialize

        del collections['collection_id']
        del collections['creation_time']
        
        if not entries:
            return jsonify({'message' :'No such record found in: ' + collection + '/' + collection_id})
        
        collections['country'] = entries[0]['country']
        collections['year'] = entries[0]['date']
        collections['value'] = entries[0]['value']
        
        
        return jsonify(collections)

In [ ]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Mar/2019 17:11:08] "POST /test HTTP/1.1" 201 -
127.0.0.1 - - [14/Mar/2019 17:11:31] "GET /test/e7ce9572-9da5-4025-b1b7-f6f8f51584da/2018/malawi HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2019 17:11:42] "GET /test/e7ce9572-9da5-4025-b1b7-f6f8f51584da/2018/malawi HTTP/1.1" 200 -


In [ ]:
# # data = GetData().post('NY.GDP.MKTP.CD')

# session = requests.Session()
# session.trust_env = False

# # session.get('http://localhost:5000/')

# resp = requests.post('http://10.190.0.76:5000/', data = {'indicator_id':'NY.GDP.MKTP.CD'})